In [4]:
import os
import requests
import pandas as pd

# keys are stored in env vars to be hidden from users
api_key = "secret_eR4tFTbg7JicddiQykrLsuQsGhTJpwPkdpl90jlrfhD";
database_id = "e05da5d0a6444566a167aff89c2ffff9";

# define request to Notion API
headers = {
    "Authorization": f"Bearer {api_key}",
    "Notion-Version": "2021-08-16",
    "Content-Type": "application/json",
}

# load first page
response = requests.post(
    f"https://api.notion.com/v1/databases/{database_id}/query", headers=headers
).json()
response

{'object': 'list',
 'results': [{'object': 'page',
   'id': '20dcdf68-a439-40e0-b638-e5a10506ac8c',
   'created_time': '2023-11-29T18:21:00.000Z',
   'last_edited_time': '2023-11-29T18:32:00.000Z',
   'created_by': {'object': 'user',
    'id': 'f3f6f9e5-3e9d-420a-bbd6-f3a504ff6791'},
   'last_edited_by': {'object': 'user',
    'id': 'f3f6f9e5-3e9d-420a-bbd6-f3a504ff6791'},
   'cover': None,
   'icon': None,
   'parent': {'type': 'database_id',
    'database_id': 'e05da5d0-a644-4566-a167-aff89c2ffff9'},
   'archived': False,
   'properties': {'project': {'id': 'Y%5D%3Be',
     'type': 'relation',
     'relation': [],
     'has_more': False},
    'Tags': {'id': 'akFy', 'type': 'multi_select', 'multi_select': []},
    'Name': {'id': 'title',
     'type': 'title',
     'title': [{'type': 'text',
       'text': {'content': 'DJ-4P', 'link': None},
       'annotations': {'bold': False,
        'italic': False,
        'strikethrough': False,
        'underline': False,
        'code': False,


In [ ]:

# iteratively load all pages
records = response["results"]
while response["has_more"]:
    response = requests.post(
        f"https://api.notion.com/v1/databases/{database_id}/query",
        json={"start_cursor": response["next_cursor"]},
        headers=headers,
    ).json()
    records += response["results"]


def extract_all_property_values(page_data):
    """
    Extracts values for all properties from a Notion page object.

    :param page_data: A dictionary representing a Notion page object.
    :return: A dictionary with property names and their corresponding values.
    """
    properties = page_data.get("properties", {})
    values = {}

    for name, prop in properties.items():
        prop_type = prop.get("type")

        if prop_type == "checkbox":
            values[name] = prop.get("checkbox")
        elif prop_type == "multi_select":
            values[name] = [item["name"] for item in prop.get("multi_select", [])]
        elif prop_type == "formula":
            values[name] = prop["formula"].get(prop["formula"]["type"])
        elif prop_type == "date":
            values[name] = prop["date"]
        elif prop_type == "title":
            values[name] = "".join([text["plain_text"] for text in prop["title"]])
        else:
            values[name] = "Unsupported property type"

    return values


all_values = [extract_all_property_values(record) for record in records]
df = pd.DataFrame(all_values)

df["due_date"] = df["due"].apply(lambda x: x["start"] if x else None)
df["due_date"] = pd.to_datetime(df["due_date"])

df